# We need to map the objects of the image into existing ontology, steps for mapping:


In [81]:
#pip install owlready2
# pip install pyyaml

Load an ontology from a local repository, or from Internet:

In [15]:
#pip install -U sentence-transformers

# 1.finding the suitable ontology for our objects (here, we select the mason ontology)

In [1]:

from owlready2 import *
from sentence_transformers import SentenceTransformer, util
from yaml.loader import SafeLoader
import yaml
from neo4j import GraphDatabase
import json
# from datetime import datetime
import torch
import glob

onto_path.append("")
onto = get_ontology("E://Jacobs 3rd sem//advproj2//main folder//mason.owl")
onto.load()

get_ontology("http://www.owl-ontologies.com/mason.owl#")

# 2.extract the list of ontology classes using owlready2

In [2]:
def get_ontoCls():
    return list(onto.classes())

In [3]:
get_ontoCls()

[mason.Electropolishing,
 mason.Anodic_dissolving,
 mason.Addition,
 mason.Mechanical_Small_variations_of_volume,
 mason.Water_trimming_machine,
 mason.Shearing_machine-tool,
 mason.Counterbore,
 mason.Revolution_entity,
 mason.Founding_machine,
 mason.Machine-tool,
 mason.Control,
 mason.Manufacturing_operation,
 mason.Tap,
 mason.Forming_machine,
 mason.Punching,
 mason.Cutting,
 mason.Polishing,
 mason.Abrasion,
 mason.Complex_shape_of_revolution_with_draught,
 mason.Complex_shape_of_revolution,
 mason.Electrolytic_ripping,
 mason.Maintenance,
 mason.Logistic_operation,
 mason.Rolling_mill,
 mason.Slot,
 mason.Linear_entity,
 mason.Assembly,
 mason.Machining_centre_5_axis,
 mason.Circular_slot,
 mason.Oxidation,
 mason.Physicochemical_Loss_of_volume,
 mason.Operation,
 mason.Manufacturing_concept,
 mason.Forging,
 mason.Brutal_hot_forming,
 mason.Forging_die_and_punch,
 mason.Tool,
 mason.Administrative_entity,
 mason.Entity,
 mason.Set,
 mason.Assembly_entity,
 mason.Cost_entity,
 

In [4]:
# onto.Machine_resource.descendants()
onto.Machine_resource.subclasses()

<generator object EntityClass.subclasses at 0x000001FB55427DD0>

In [4]:
def open_saveYaml(path_yaml):
    with open(path_yaml) as f:
        data=yaml.load(f, Loader=SafeLoader)
        #to get the list of names to extract classes/instances from it
        obj_list=data.get('names')
        return obj_list
        
path_="E://Jacobs 3rd sem//advproj2//main folder//data.yaml"
yamldata=open_saveYaml(path_)
yamldata

['Printer3D_Body',
 'Printer3D_Filament',
 'BandSaw_Controls',
 'BandSaw_Saw',
 'BandSaw_WorkTable',
 'Conveyors_BaseRoller',
 'Conveyors_Bed',
 'Conveyors_SlantRoller',
 'Drillpress_Controls',
 'Drillpress_Drill',
 'Drillpress_HandWheel',
 'Drillpress_MachineStand',
 'Drillpress_WorkTable',
 'Lathe_Bed',
 'Lathe_ChipPan',
 'Lathe_Controls',
 'Lathe_CrossSlide',
 'MillingMachine_ChipCollectorPlate',
 'MillingMachine_Controls',
 'MillingMachine_HandWheel',
 'MillingMachine_MachineStand',
 'MillingMachine_WorkTable',
 'Robot_BaseArm',
 'Robot_Head',
 'Robot_UpperArm']

In [5]:
#to get only classes, split each names and get the first word before '_'. Used 'set' to get the distinct vales of classes
def get_clsYaml(yamldata):
    cls_lst = set([x.split('_')[0] for x in yamldata])
    class_list=[]                                                      #label list
    #to make a list of classes.
    for s in cls_lst: class_list.append(s) 
    return class_list
    
class_list=get_clsYaml(yamldata)
class_list

['Drillpress',
 'BandSaw',
 'Printer3D',
 'Conveyors',
 'MillingMachine',
 'Robot',
 'Lathe']

# 3. finding the related classes for our objects using similarity measures .
# If there are no matches classes for our object, we will add the class to our ontology using owlready2

In [6]:
def createNewClass(createClass,s_class):
    y = createClass
    s_class=s_class
    if s_class=='Thing':
        with onto:
            SuperClass =  onto.Thing #static value (root class)
            NewClass = types.new_class(y, (Thing,))
            print(y, s_class)
#     else:
#         SuperClass =  s_class #static value (root class)
#         NewClass = types.new_class(y,(onto[s_class],))
#         print('out of thing: ' ,y ,s_class)

In [7]:

"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 3 most similar sentences in the corpus.
"""
def mat_cls_onto(queries):
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Corpus with example sentences
    cls_lst_onto = get_ontoCls()
    cls_str_onto = []
    for n in cls_lst_onto:
        cls_str_onto.append(str(n))
    #print(abc_str)


    corpus = cls_str_onto
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    cls_dict ={}
    # Query sentences:
    # queries = class_list
    # Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(3, len(corpus))
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)

        c_lst=[]
        scr_lst=[]
        
        # We use cosine-similarity and torch.topk to find the highest 3 scores
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)
    #     print(top_results)
        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 3 most similar sentences in corpus:")

        for score, idx in zip(top_results[0], top_results[1]):
            print('c_lst ',corpus[idx], "(Score: {:.4f})".format(score))
            c_lst.append(corpus[idx].replace('mason.', ''))
            scr_lst.append("{:.4f}".format(score)) 
    #         print('c_lst ',c_lst)

        if float(scr_lst[0])>=0.5:
    #         print("please select super_class number: 1- "+c_lst[0]+ ". 2- " + c_lst[1] + ". 3- " + c_lst[2])
    #         print("score:"+ str(scr_lst[0]) )
            user_input = int(input("please select super_class number: 1- "+c_lst[0]+ ". 2- " + c_lst[1] + ". 3- " + c_lst[2]))

            if user_input==1:
                spr_cls = c_lst[0]
                
                cls_dict[query]=spr_cls
                
                print(cls_dict, query,spr_cls)
#                    

            if user_input==2:
                spr_cls = c_lst[1]
                cls_dict[query]=spr_cls
                
                print(cls_dict, query,spr_cls)
#                     

            if user_input==3:
                spr_cls = c_lst[2]
                cls_dict[query]=spr_cls
                print(cls_dict, query,spr_cls)
#                    

        elif float(scr_lst[0])<0.5:
            spr_cls='Thing'
            print('outside func ',query, spr_cls)
            createNewClass(query,spr_cls)
            
    return cls_dict

cls_dict=mat_cls_onto(class_list)  
cls_dict
            
#             class Images(onto.Machine_resource):
#                 namespace = onto
            
"""
# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
    print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
"""





Query: Drillpress

Top 3 most similar sentences in corpus:
c_lst  mason.Drilling (Score: 0.5551)
c_lst  mason.Drill (Score: 0.5074)
c_lst  mason.Machine-tool (Score: 0.4659)
please select super_class number: 1- Drilling. 2- Drill. 3- Machine-tool1
{'Drillpress': 'Drilling'} Drillpress Drilling




Query: BandSaw

Top 3 most similar sentences in corpus:
c_lst  mason.Board_drop_hammer (Score: 0.2988)
c_lst  mason.Welding_aluminium (Score: 0.2937)
c_lst  mason.Shearing_machine-tool (Score: 0.2744)
outside func  BandSaw Thing
BandSaw Thing




Query: Printer3D

Top 3 most similar sentences in corpus:
c_lst  mason.Plasma_trimming_machine (Score: 0.2798)
c_lst  mason.Laser (Score: 0.2543)
c_lst  mason.Machining_centre_5_axis (Score: 0.2481)
outside func  Printer3D Thing
Printer3D Thing




Query: Conveyors

Top 3 most similar sentences in corpus:
c_lst  mason.Hydroforming (Score: 0.3268)
c_lst  mason.Manufacturing_concept (Score: 0.3115)
c_lst  mason.Manual_moulding (Score: 0.3106)
outsi

'\n# Alternatively, we can also use util.semantic_search to perform cosine similarty + topk\nhits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)\nhits = hits[0]      #Get the hits for the first query\nfor hit in hits:\n    print(corpus[hit[\'corpus_id\']], "(Score: {:.4f})".format(hit[\'score\']))\n'

In [8]:
cls_dict

{'Drillpress': 'Drilling',
 'MillingMachine': 'Milling_machine',
 'Lathe': 'Lathe'}

# 6. Extract the objects from images dynamically.


In [ ]:
# first-extract json files data

In [9]:

# Place your JSON data in a directory named 'data/'
def getImg_objs():
    src = "data/"
#     date = datetime.now()
    data = []

    # Change the glob if you want to only look through files with specific names
    files = glob.glob('data/*', recursive=True)
    print('file is:',files)

    for single_file in files:

    #     print(single_file)
        with open(single_file, 'r') as f:

            # Use 'try-except' to skip files that may be missing data
            try:
                json_file = json.load(f)
                data.append([
                  json_file['img_id'],
                  json_file['img_name'],          
                  json_file['detection']
                ])
            except KeyError:
                print(f'Skipping {single_file}')

    # Sort the data
    data.sort()
    return data

data = getImg_objs()

file is: ['data\\IMG_0209.json', 'data\\IMG_0391.json', 'data\\IMG_0893.json']


In [50]:
# data[0]

# 7. creating and populating instances of respective classes dynamically.

In [10]:
def create_pop(data):
    for i in range(0,len(data)):
        d_len=len(data[i][2])
        for j in range(0,d_len):
            img_obj = data[i][2][j].get('det_name').split('_')[0]
#             print(img_obj)
            if img_obj  not in cls_dict:
                inst  = onto[img_obj](str(data[i][2][j]))
                print(str(data[i][2][j]))
            elif img_obj in cls_dict:
                cls_dict_val = cls_dict.get(img_obj)
                inst  = onto[cls_dict_val](str(data[i][2][j]))
                print(str(data[i][2][j]),) 
                
create_pop(data)                        

{'det_cat': 0, 'det_name': 'Printer3D_Body', 'det_x': 0.6924999952316284, 'det_y': 0.5791249871253967, 'det_w': 0.12966667115688324, 'det_h': 0.3877499997615814, 'det_score': 0.2528335154056549}
{'det_cat': 0, 'det_name': 'Printer3D_Body', 'det_x': 0.4426666796207428, 'det_y': 0.625374972820282, 'det_w': 0.0860000029206276, 'det_h': 0.132750004529953, 'det_score': 0.301304429769516}
{'det_cat': 0, 'det_name': 'Printer3D_Body', 'det_x': 0.7568333148956299, 'det_y': 0.6306250095367432, 'det_w': 0.08733333647251129, 'det_h': 0.12825000286102295, 'det_score': 0.3674049377441406}
{'det_cat': 1, 'det_name': 'Printer3D_Filament', 'det_x': 0.8255000114440918, 'det_y': 0.6620000004768372, 'det_w': 0.07199999690055847, 'det_h': 0.04650000110268593, 'det_score': 0.4139844477176666}
{'det_cat': 1, 'det_name': 'Printer3D_Filament', 'det_x': 0.7668333053588867, 'det_y': 0.6194999814033508, 'det_w': 0.04800000041723251, 'det_h': 0.14000000059604645, 'det_score': 0.45336398482322693}
{'det_cat': 0, 'd

In [11]:
 # To check for single class:
for i in onto.Milling_machine.instances(): 
    print(i)

mason.{'det_cat': 21, 'det_name': 'MillingMachine_WorkTable', 'det_x': 0.48091667890548706, 'det_y': 0.6917499899864197, 'det_w': 0.20283333957195282, 'det_h': 0.1509999930858612, 'det_score': 0.3188326954841614}
mason.{'det_cat': 18, 'det_name': 'MillingMachine_Controls', 'det_x': 0.47608333826065063, 'det_y': 0.37325000762939453, 'det_w': 0.10483333468437195, 'det_h': 0.2919999957084656, 'det_score': 0.4383426606655121}


# 8. Saving ontology

In [12]:
##save ontology
onto.save()
try:
    onto.save(file = "./ontology_mason.rdf", format = "rdfxml")
except ValueError:
    print(ValueError)
# onto.save(file = "test_mason", format = "rdfxml")

# 9. import the final ontology into Neo4j

In [39]:
# !pip install neo4j 

In [16]:
#Upload ontology to Neo4j for graphing 

class Neo4j_connection: #neo4jconnection #used to be COnnect2Neo4j

    def __init__(self, uri, user, pwd, database_name="neo4j"): #connect to the server; create constraint if not exist; default database is "Neo4j"
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        self.__database_name = database_name

        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create driver:", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        CALL db.constraints()
                 '''
        a = self.query(query_string, db= self.__database_name)
        constr = str(a)
        try:
            b = constr[constr.find("description='")+13:constr.find(' ON')]
        except Exception as e:
            pass
        if b == 'CONSTRAINT':
            print("Constraint already existed")
        else:
            try:
                query_string = '''
                    CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE  
                            '''
                self.query(query_string, db = self.__database_name)
            except Exception as e:
                print("Failed to create constraint - check again", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        match(n) return count(n)
        '''
        a = self.query(query_string, db= self.__database_name)
        strin = str(a[0])
        config_check = int(strin[strin.find("=")+1:strin.find(">")])
        if config_check != 0:
            print("Config already existed")
        else:
            try: #setting up neccessary config for neosemantics
                query_string = '''
                    CALL n10s.graphconfig.init()
                                '''
                self.query(query_string, db= self.__database_name)

            except Exception as e:
                print("Failed to initiate graphconfig: ", e)


    def uploading_orx(self, address, db="neo4j"): #uploading OWL, RDF, and XML #i am trying to
        query_string = (
                "call n10s.rdf.import.fetch('"+address+"','RDF/XML',"+"{"+"verifyUriSyntax: false"+"}"+")")
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded XML/RDF/OWL file ")
        except Exception as e:
            print("Upload failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def uploading_ttl(self, address, db=None): #uploading Turtle files
        query_string = (
                "call n10s.rdf.import.fetch(" + f'{address}'+  ',"Turtle",{verifyUriSyntax: false})')
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded TTL file ")
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None): #query commands of choice.
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

a = Neo4j_connection("bolt://localhost:7687", "neo4j", "password") #("bolt://localhost:7687", "neo4j", "password")
a.uploading_orx("file:///Users/ABHISHEK MATHUR/ontology_mason.rdf", "neo4j")


Query failed: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `db.constraints` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}
Query failed: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent constraint already exists, 'Constraint( id=4, name='n10s_unique_uri', type='UNIQUENESS', schema=(:Resource {uri}), ownedIndex=3 )'.}
Config already existed
Sucessfully uploaded XML/RDF/OWL file 


[<Record terminationStatus='OK' triplesLoaded=1588 triplesParsed=1588 namespaces={'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'owl': 'http://www.w3.org/2002/07/owl#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 'ns0': 'http://www.owl-ontologies.com/mason.owl#'} extraInfo='' callParams={'verifyUriSyntax': False}>]

# Important functions of owlready2

In [32]:
#Access the classes of the ontology, and create new instances / individuals:
# onto.Thing
# list(onto.object_properties())
# list(onto.data_properties())
# list(onto.annotation_properties())
# list(onto.properties())
# list(onto.disjoint_classes() )
# list(onto.disjoint_properties())
# list(onto .disjoints() )
# list(onto.different_individuals())
# print(onto.Lathe.is_a)  #to get parent class
# destroy_entity(onto.Images)
# Owlready classes and individuals can be used as values within search(), as follows:
# onto.search_one(is_a=onto.Images)
#to check the flow of parent and subclasses # .descendants()#.ancestors()
# Robots.descendants() 